In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
pd.set_option("display.max_rows", 120)

dados_treino = pd.read_csv(
    "/home/pinguim/Documentos/MeusEstudos/QuarentenaDados/DADOS_TREINO.csv")
dados_teste = pd.read_csv(
    "/home/pinguim/Documentos/MeusEstudos/QuarentenaDados/DADOS_TESTE.csv")
dados_desafioqt = pd.read_csv(
    "/home/pinguim/Documentos/MeusEstudos/QuarentenaDados/DESAFIOQT.csv")

assert dados_treino.shape == (150000, 5)
assert dados_teste.shape == (20000, 5)
assert dados_desafioqt.shape == (10000, 5)

In [28]:
coluna_label = 'NU_NOTA_LC'
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()
X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_treino = sc.fit_transform(X_treino)  
X_teste = sc.transform(X_teste)


In [62]:
from sklearn.ensemble import RandomForestRegressor
modelo_rf = RandomForestRegressor(
    criterion='mse',
    max_depth=10,
    max_features=3,
    max_leaf_nodes=None,
    min_impurity_split=None,
    min_samples_leaf=3,
    min_samples_split=5,
    min_weight_fraction_leaf=0.0,
    n_estimators=600,
    n_jobs=1,
    random_state=0,
    verbose=0,
    warm_start=False
)

In [59]:
modelo_rf.fit(X_treino, Y_treino)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features=3, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [60]:
y_pred_treino = modelo_rf.predict(X_treino)
y_pred_teste = modelo_rf.predict(X_teste)

In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

print(cross_val_score(modelo_rf, X_treino, Y_treino, cv = 5))
print(cross_val_score(modelo_rf, X_teste, Y_teste, cv = 5))
print(mean_squared_error(Y_treino, y_pred_treino).round(2))
print(mean_squared_error(Y_teste, y_pred_teste).round(2))

[0.6143602  0.61213477 0.61189022 0.61666617 0.61030214]
[0.59493789 0.59465658 0.59564782 0.59754766 0.60666248]
1889.55
2057.2


In [65]:
from sklearn.neural_network import MLPRegressor

modelo_mlp = MLPRegressor(
    random_state=0, hidden_layer_sizes=100, activation='tanh',validation_fraction=0.1230,  max_iter=1000)

In [69]:
modelo_mlp.fit(X_treino, Y_treino)

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=100, learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=1000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=0, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.123, verbose=False,
             warm_start=False)

In [71]:
y_pred_teste_1 = modelo_mlp.predict(X_teste)
y_pred_train_1 = modelo_mlp.predict(X_treino)

In [72]:
print(cross_val_score(modelo_mlp, X_teste, Y_teste, cv = 5))
mean_squared_error(Y_teste, y_pred_teste_1).round(2)

[0.60086777 0.59722299 0.59948319 0.59671748 0.61439074]


2045.21

In [16]:
from sklearn.neighbors import KNeighborsRegressor

modelo_knr = KNeighborsRegressor()

In [17]:
modelo_knr.fit(X_treino, Y_treino)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [18]:
y_pred_teste_2 = modelo_knr.predict(X_teste)
y_pred_train_2 = modelo_knr.predict(X_treino)

In [19]:
print(cross_val_score(modelo_knr, X_teste, Y_teste, cv = 5))
mean_squared_error(Y_teste, y_pred_teste_2).round(2)

[0.52082818 0.51683287 0.51751788 0.50936585 0.54839745]


2471.08

In [109]:
import lightgbm as lgbm
modelo_lgbm = lgbm.sklearn.LGBMRegressor(
    n_estimators=100, random_state=0, learning_rate=0.052339, boosting_type='gbdt', n_jobs=-1)

In [110]:
modelo_lgbm.fit(X_treino, Y_treino)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.052339, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [111]:
y_pred_teste_3 = modelo_lgbm.predict(X_teste)
y_pred_train_3 = modelo_lgbm.predict(X_treino)

In [112]:
print(cross_val_score(modelo_lgbm, X_teste, Y_teste, cv = 5))
mean_squared_error(Y_teste, y_pred_teste_3).round(2)

[0.59013343 0.59723217 0.59256695 0.60086626 0.6019208 ]


2079.9

In [189]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

#print("Parameter optimization")
modelo_xgb = xgb.XGBRegressor(
    n_estimators=20, max_depth=4, reg_lambda=2, min_child_weight=3, subsample=1, colsample_bytree=1, random_state=0)
# reg_xgb = GridSearchCV(modelo_xgb,
#                   {'max_depth': [2,4,6],
#                    'n_estimators': [50,100,200]}, verbose=1)


modelo_xgb.fit(X_treino, Y_treino)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=4,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             n_estimators=20, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=2, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [190]:
y_pred_teste_4 = modelo_xgb.predict(X_teste)
y_pred_train_4 = modelo_xgb.predict(X_treino)

In [191]:
print(cross_val_score(modelo_xgb, X_teste, Y_teste, cv = 5))
mean_squared_error(Y_teste, y_pred_teste_4).round(2)

[0.59622723 0.59326663 0.59636238 0.59481403 0.60597046]


2061.18